In [11]:
import scipy.io as scio
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cv2
from scipy import ndimage
from matplotlib import pyplot
import scipy
from PIL import Image
from skimage import measure
import math
from tqdm import tqdm_notebook as tqdm
import os

In [20]:
cd U:\Siyan\synEM_train_data

U:\Siyan\synEM_train_data


In [23]:
trainData = os.listdir('U:\Siyan\synEM_train_data')

In [31]:
trainData

['2012-09-28_ex145_07x2_mag1_x501-1000_y1001-1500_z501-700.mat',
 '2012-09-28_ex145_07x2_mag1_x2001-2500_y3001-3500_z801-1000.mat',
 '2012-09-28_ex145_07x2_mag1_x1001-1500_y1001-1500_z3000-3200.mat',
 '2012-09-28_ex145_07x2_mag1_x1501-2000_y1001-1500_z2001-2200.mat',
 '2012-09-28_ex145_07x2_mag1_x1501-2000_y501-1000_z2001-2200.mat',
 '2012-09-28_ex145_07x2_mag1_x3001-3500_y3001-3500_z1401-1600.mat',
 '2012-09-28_ex145_07x2_mag1_x1501-2000_y1001-1500_z1001-1200.mat',
 '2012-09-28_ex145_07x2_mag1_x3001-3500_y3001-3500_z1001-1200.mat',
 '2012-09-28_ex145_07x2_mag1_x3001-3500_y3001-3500_z1201-1400.mat',
 '2012-09-28_ex145_07x2_mag1_x3501-4000_y1501-2000_z1001-1200.mat',
 '2012-09-28_ex145_07x2_mag1_x1501-2000_y1501-2000_z2001-2200.mat',
 '2012-09-28_ex145_07x2_mag1_x2001-2500_y1001-1500_z1001-1200.mat',
 '2012-09-28_ex145_07x2_mag1_x4001-4500_y1001-1500_z801-1000.mat',
 '2012-09-28_ex145_07x2_mag1_x2001-2500_y2001-2500_z1901-2100.mat',
 '2012-09-28_ex145_07x2_mag1_x1001-1500_y501-1000_z500

In [24]:
len(trainData)

40

In [30]:
trainData[22]

'2012-09-28_ex145_07x2_mag1_x735-1585_y980-1581_z769-1065.mat'

In [34]:
f = h5py.File('2012-09-28_ex145_07x2_mag1_x735-1585_y980-1581_z769-1065.mat', 'r')

In [40]:
f['data']['raw'].shape

(297, 602, 851)

In [29]:
for trainNo in tqdm(range(len(trainData))[22:23]):
    train1 = scio.loadmat(trainData[trainNo])
    
    #raw
    raw = train1['data']['raw'][0][0]
    raw_ = np.moveaxis(raw, -1, 0)
    h5f = h5py.File('synEM_train_raw_'+str('%03d'%trainNo)+'.h5', 'w')
    h5f.create_dataset('main', data=raw_)
    h5f.close()
    
    #synapse direction
    X = raw.shape[0]
    Y = raw.shape[1]
    Z = raw.shape[2]
    interfaceLabels = train1['interfaceLabels']
    synapse1 = np.where(interfaceLabels == 1)[0]
    synapse2 = np.where(interfaceLabels == 2)[0]
    synapse = np.hstack((synapse1,synapse2))
    
    #interface
    interface = np.zeros((raw.shape),dtype='uint8')
    interfaceSurfaceList = train1['data']['interfaceSurfaceList'][0][0]
    for i in synapse:
        interfacei = interfaceSurfaceList[i][0]
        for j in range(interfacei.shape[0]):
            idx = interfacei[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            interface[x,y,z]=255
            
    #subvolume
    Polarity = train1['data']['subsegmentsList'][0][0][0]
    smallP = Polarity[0]
    midP = Polarity[1]
    largeP = Polarity[2]
    
    smallPolarity = np.zeros((raw.shape),dtype = 'uint8')
    for i in synapse1:
        prei = smallP[i][0]
        posti = smallP[i][1]
        for j in range(prei.shape[0]):
            idx = prei[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            smallPolarity[x,y,z]=255
        for j in range(posti.shape[0]):
            idx = posti[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            smallPolarity[x,y,z]=128
            
    for i in synapse2:
        posti = smallP[i][0]
        prei = smallP[i][1]
        for j in range(prei.shape[0]):
            idx = prei[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            smallPolarity[x,y,z]=255
        for j in range(posti.shape[0]):
            idx = posti[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            smallPolarity[x,y,z]=128
            
    midPolarity = np.zeros((raw.shape),dtype = 'uint8')
    for i in synapse1:
        prei = midP[i][0]
        posti = midP[i][1]
        for j in range(prei.shape[0]):
            idx = prei[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            midPolarity[x,y,z]=255
        for j in range(posti.shape[0]):
            idx = posti[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            midPolarity[x,y,z]=128
            
    for i in synapse2:
        posti = midP[i][0]
        prei = midP[i][1]
        for j in range(prei.shape[0]):
            idx = prei[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            midPolarity[x,y,z]=255
        for j in range(posti.shape[0]):
            idx = posti[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            midPolarity[x,y,z]=128
            
    largePolarity = np.zeros((raw.shape),dtype = 'uint8')
    for i in synapse1:
        prei = largeP[i][0]
        posti = largeP[i][1]
        for j in range(prei.shape[0]):
            idx = prei[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            largePolarity[x,y,z]=255
        for j in range(posti.shape[0]):
            idx = posti[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            largePolarity[x,y,z]=128
            
    for i in synapse2:
        posti = largeP[i][0]
        prei = largeP[i][1]
        for j in range(prei.shape[0]):
            idx = prei[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            largePolarity[x,y,z]=255
        for j in range(posti.shape[0]):
            idx = posti[j][0]
            z = math.floor(idx/(X*Y))
            y = math.floor((idx-z*X*Y)/X)
            x = idx-z*X*Y - y*X -1
            largePolarity[x,y,z]=128
            
    #label generating
    Label = np.zeros((4,raw.shape[2],raw.shape[0],raw.shape[1]))
    Label[0]=np.moveaxis(interface, -1, 0)
    Label[1]=np.moveaxis(smallPolarity, -1, 0)
    Label[2]=np.moveaxis(midPolarity, -1, 0)
    Label[3]=np.moveaxis(largePolarity, -1, 0)
    h5f = h5py.File('synEM_train_label_'+str('%03d'%trainNo)+'.h5', 'w')
    h5f.create_dataset('main', data=Label)
    h5f.close()

NotImplementedError: Please use HDF reader for matlab v7.3 files